In [55]:
import itertools as it

import networkx as nx
import numpy as np
from scipy import sparse

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data


In [56]:
class GCN(torch.nn.Module):
    def __init__(self, node_features):
        super().__init__()
        self.conv1 = GCNConv(node_features, 64)
        self.conv2 = GCNConv(64, 32)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)

        return x


In [57]:
# utils 

def read_sat(sat_path):
    with open(sat_path) as f:
        sat_lines = f.readlines()
        header = sat_lines[0]
        header_info = header.replace("\n", "").split(" ")
        num_vars = int(header_info[-2])
        num_clauses = int(header_info[-1])

        sat = [[int(x) for x in line.replace(' 0\n', '').split(' ')]
               for line in sat_lines[1:]]

        return sat, num_vars, num_clauses


def sat_to_lig_adjacency_matrix(sat, num_vars):
    get_literal_idx = lambda x: 2 * x - 2 if x > 0 else 2 * abs(x) - 1
    lig_adjacency_matrix = np.zeros([2*num_vars, 2*num_vars])
    lig_weighted_adjacency_matrix = np.zeros([2*num_vars, 2*num_vars])

    for clause in sat:
        pairs = it.combinations(clause, 2)
#         print(f'clause: {clause}')
        for pair in pairs:
            x_idx = get_literal_idx(pair[0])
            y_idx = get_literal_idx(pair[1])
#             print(f'pair: {(x_idx, y_idx)}')
            lig_adjacency_matrix[x_idx, y_idx] = 1
            lig_adjacency_matrix[y_idx, x_idx] = 1
            lig_weighted_adjacency_matrix[x_idx, y_idx] += 1
            lig_weighted_adjacency_matrix[y_idx, x_idx] += 1    
    return lig_adjacency_matrix, lig_weighted_adjacency_matrix

In [58]:
sat_path = './ssa2670-141.processed.cnf'
sat_instance, num_vars, num_clauses = read_sat(sat_path)

lig_adjacency_matrix, lig_weighted_adjacency_matrix = sat_to_lig_adjacency_matrix(sat_instance, num_vars)

# graph = nx.from_numpy_matrix(lig_adjacency_matrix)
# edges = nx.to_edgelist(graph)
# print(lig_adjacency_matrix.nonzero())

edge_index = torch.tensor(lig_adjacency_matrix.nonzero(), dtype=torch.long)
edge_value = lig_weighted_adjacency_matrix[lig_adjacency_matrix.nonzero()]
print(edge_value)
max_edge_value = max(edge_value)
norm_edge_value = edge_value/max_edge_value
print(norm_edge_value)
embeddings = torch.load('./embeddings.pt')
embeddings.requires_grad = False
# print(embeddings)
x = embeddings

data = Data(x=x, edge_index=edge_index)


[6. 6. 1. ... 2. 1. 3.]
[1.         1.         0.16666667 ... 0.33333333 0.16666667 0.5       ]


In [59]:
lig_adjacency_matrix
sparse_matrix = sparse.csr_matrix(lig_adjacency_matrix)
sparse_matrix

<182x182 sparse matrix of type '<class 'numpy.float64'>'
	with 2124 stored elements in Compressed Sparse Row format>

In [60]:
# CELL
import warnings
warnings.filterwarnings('ignore')

import pickle
import numpy as np
import scipy.sparse as sp
from scipy.sparse import load_npz

import torch

from cell.utils import link_prediction_performance
from cell.cell import Cell, EdgeOverlapCriterion, LinkPredictionCriterion
from cell.graph_statistics import compute_graph_statistics

model = Cell(A=sparse_matrix,
             H=9,
             callbacks=[EdgeOverlapCriterion(invoke_every=10, edge_overlap_limit=.5)])


model.train(steps=200,
            optimizer_fn=torch.optim.Adam,
            optimizer_args={'lr': 0.1,
                            'weight_decay': 1e-7})


Step:  10/200 Loss: 3.93387 Edge-Overlap: 0.402 Total-Time: 0
Step:  20/200 Loss: 3.36424 Edge-Overlap: 0.546 Total-Time: 0


In [61]:
generated_graph = model.sample_graph()
print(generated_graph.A)
print(generated_graph.A.nonzero())
# compute_graph_statistics(generated_graph)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(array([  0,   0,   0, ..., 181, 181, 181]), array([  2,  14,  18, ..., 134, 169, 170]))


In [62]:
# training
model = GCN(50)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# print(norm_edge_value)
model.train()
for epoch in range(20):
    optimizer.zero_grad()
    out = model(data)
    src, dst = edge_index
    score = (out[src] * out[dst]).sum(dim=-1)
    score = torch.sigmoid(score)
    loss = F.mse_loss(score, torch.tensor(norm_edge_value, dtype=torch.float))
    print(f'epoch: {epoch}, loss: {loss.item()}')
    loss.backward()
    optimizer.step()


epoch: 0, loss: 0.29498571157455444
epoch: 1, loss: 0.14215028285980225
epoch: 2, loss: 0.11060117930173874
epoch: 3, loss: 0.1124689131975174
epoch: 4, loss: 0.11748021841049194
epoch: 5, loss: 0.11742106080055237
epoch: 6, loss: 0.11301187425851822
epoch: 7, loss: 0.10726536065340042
epoch: 8, loss: 0.10215672105550766
epoch: 9, loss: 0.09826977550983429
epoch: 10, loss: 0.09558629989624023
epoch: 11, loss: 0.09383457899093628
epoch: 12, loss: 0.09270072728395462
epoch: 13, loss: 0.0919472724199295
epoch: 14, loss: 0.09142032265663147
epoch: 15, loss: 0.0910167470574379
epoch: 16, loss: 0.09069794416427612
epoch: 17, loss: 0.09044041484594345
epoch: 18, loss: 0.09022955596446991
epoch: 19, loss: 0.09005921334028244


In [67]:
out = model(data)
src, dst = edge_index
score = (out[src] * out[dst]).sum(dim=-1)
print(max(score))

tensor(0.0586, grad_fn=<UnbindBackward0>)


In [66]:
graph_prime = generated_graph.A
edge_index_prime = torch.tensor(graph_prime.nonzero(), dtype=torch.long)
print(edge_index_prime.size())
data_prime = Data(x=x, edge_index = edge_index_prime)
out = model(data_prime)
print(out)
src, dst = edge_index_prime
score = (out[src] * out[dst]).sum(dim=-1)
print(max(score))
score = torch.sigmoid(score)


torch.Size([2, 2124])
tensor([[ 0.0053, -0.0137, -0.0285,  ...,  0.0019,  0.0034,  0.0080],
        [ 0.0302,  0.0202,  0.0091,  ...,  0.0299, -0.0138,  0.0040],
        [-0.0145, -0.0196,  0.0104,  ..., -0.0133, -0.0064, -0.0068],
        ...,
        [-0.0074,  0.0216, -0.0311,  ...,  0.0071,  0.0219,  0.0137],
        [ 0.0012,  0.0150,  0.0023,  ...,  0.0222,  0.0061, -0.0085],
        [-0.0163,  0.0131,  0.0322,  ...,  0.0327, -0.0088, -0.0250]],
       grad_fn=<AddBackward0>)
tensor(0.0667, grad_fn=<UnbindBackward0>)


In [65]:
print(score)

tensor([0.5009, 0.5008, 0.5014,  ..., 0.5033, 0.5044, 0.5025],
       grad_fn=<SigmoidBackward0>)
